# 使用 OpenAI Functions 构建回答结构

OpenAI Functions 允许对响应输出进行结构化。这在问答场景中特别有用，您不仅可以获取最终答案，还可以获取支持证据、引用等。

在本教程中，我们将展示如何在整体检索管道中使用包含 OpenAI Functions 的 LLM 链。

In [25]:
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [26]:
loader = TextLoader("../../state_of_the_union.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
for i, text in enumerate(texts):
    text.metadata["source"] = f"{i}-pl"
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [27]:
from langchain.chains import create_qa_with_sources_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [28]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [29]:
qa_chain = create_qa_with_sources_chain(llm)

In [30]:
doc_prompt = PromptTemplate(
    template="Content: {page_content}\nSource: {source}",
    input_variables=["page_content", "source"],
)

In [31]:
final_qa_chain = StuffDocumentsChain(
    llm_chain=qa_chain,
    document_variable_name="context",
    document_prompt=doc_prompt,
)

In [32]:
retrieval_qa = RetrievalQA(
    retriever=docsearch.as_retriever(), combine_documents_chain=final_qa_chain
)

In [33]:
query = "What did the president say about russia"

In [34]:
retrieval_qa.run(query)

'{\n  "answer": "The President expressed strong condemnation of Russia\'s actions in Ukraine and announced measures to isolate Russia and provide support to Ukraine. He stated that Russia\'s invasion of Ukraine will have long-term consequences for Russia and emphasized the commitment to defend NATO countries. The President also mentioned taking robust action through sanctions and releasing oil reserves to mitigate gas prices. Overall, the President conveyed a message of solidarity with Ukraine and determination to protect American interests.",\n  "sources": ["0-pl", "4-pl", "5-pl", "6-pl"]\n}'

## 使用 Pydantic

如果需要，我们可以设置链来返回 Pydantic 格式。请注意，如果下游链（包括记忆）使用此链的输出，它们通常会期望输出为字符串格式，因此您应该仅在这是最终链时使用此链。

In [35]:
qa_chain_pydantic = create_qa_with_sources_chain(llm, output_parser="pydantic")

In [36]:
final_qa_chain_pydantic = StuffDocumentsChain(
    llm_chain=qa_chain_pydantic,
    document_variable_name="context",
    document_prompt=doc_prompt,
)

In [37]:
retrieval_qa_pydantic = RetrievalQA(
    retriever=docsearch.as_retriever(), combine_documents_chain=final_qa_chain_pydantic
)

In [38]:
retrieval_qa_pydantic.run(query)

AnswerWithSources(answer="The President expressed strong condemnation of Russia's actions in Ukraine and announced measures to isolate Russia and provide support to Ukraine. He stated that Russia's invasion of Ukraine will have long-term consequences for Russia and emphasized the commitment to defend NATO countries. The President also mentioned taking robust action through sanctions and releasing oil reserves to mitigate gas prices. Overall, the President conveyed a message of solidarity with Ukraine and determination to protect American interests.", sources=['0-pl', '4-pl', '5-pl', '6-pl'])

## 在 ConversationalRetrievalChain 中使用

我们还可以展示在 ConversationalRetrievalChain 中使用它的方式。注意，由于此链涉及记忆，我们将不会使用 Pydantic 返回类型。

In [ ]:
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
_template = """基于以下对话和后续问题，将后续问题重新表述为一个独立的问题，使用原始语言。\
确保避免使用任何不明确的代词。

聊天历史：
{chat_history}
后续输入：{question}
独立问题："""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
condense_question_chain = LLMChain(
    llm=llm,
    prompt=CONDENSE_QUESTION_PROMPT,
)

In [40]:
qa = ConversationalRetrievalChain(
    question_generator=condense_question_chain,
    retriever=docsearch.as_retriever(),
    memory=memory,
    combine_docs_chain=final_qa_chain,
)

In [41]:
query = "What did the president say about Ketanji Brown Jackson"
result = qa({"question": query})

In [42]:
result

{'question': 'What did the president say about Ketanji Brown Jackson',
 'chat_history': [HumanMessage(content='What did the president say about Ketanji Brown Jackson', additional_kwargs={}, example=False),
  AIMessage(content='{\n  "answer": "The President nominated Ketanji Brown Jackson as a Circuit Court of Appeals Judge and praised her as one of the nation\'s top legal minds who will continue Justice Breyer\'s legacy of excellence.",\n  "sources": ["31-pl"]\n}', additional_kwargs={}, example=False)],
 'answer': '{\n  "answer": "The President nominated Ketanji Brown Jackson as a Circuit Court of Appeals Judge and praised her as one of the nation\'s top legal minds who will continue Justice Breyer\'s legacy of excellence.",\n  "sources": ["31-pl"]\n}'}

In [43]:
query = "what did he say about her predecessor?"
result = qa({"question": query})

In [44]:
result

{'question': 'what did he say about her predecessor?',
 'chat_history': [HumanMessage(content='What did the president say about Ketanji Brown Jackson', additional_kwargs={}, example=False),
  AIMessage(content='{\n  "answer": "The President nominated Ketanji Brown Jackson as a Circuit Court of Appeals Judge and praised her as one of the nation\'s top legal minds who will continue Justice Breyer\'s legacy of excellence.",\n  "sources": ["31-pl"]\n}', additional_kwargs={}, example=False),
  HumanMessage(content='what did he say about her predecessor?', additional_kwargs={}, example=False),
  AIMessage(content='{\n  "answer": "The President honored Justice Stephen Breyer for his service as an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court.",\n  "sources": ["31-pl"]\n}', additional_kwargs={}, example=False)],
 'answer': '{\n  "answer": "The President honored Justice Stephen Breyer for his service as an Army veteran, Constitutional scholar, and

## 使用自定义输出模式

我们可以通过传入自己的模式来更改链的输出。此模式的值和描述将影响我们传递给 OpenAI API 的函数，这不仅会影响我们如何解析输出，还会改变 OpenAI 输出本身。例如，我们可以在模式中添加 `countries_referenced` 参数并描述我们希望此参数表示的含义，这将使 OpenAI 输出在响应中包含说话者的描述。

除了前面的示例外，我们还可以向链添加自定义提示。这将允许您向响应添加额外的上下文，这对问答很有用。

In [45]:
from typing import List

from langchain.chains.openai_functions import create_qa_with_structure_chain
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

In [46]:
class CustomResponseSchema(BaseModel):
    """问题的答案，包含来源。"""

    answer: str = Field(..., description="对所问问题的回答")
    countries_referenced: List[str] = Field(
        ..., description="在来源中提到的所有国家"
    )
    sources: List[str] = Field(
        ..., description="用于回答问题的来源列表"
    )


prompt_messages = [
    SystemMessage(
        content=(
            "你是一个世界级的算法，用于以特定格式回答问题。"
        )
    ),
    HumanMessage(content="使用以下上下文回答问题"),
    HumanMessagePromptTemplate.from_template("{context}"),
    HumanMessagePromptTemplate.from_template("问题: {question}"),
    HumanMessage(
        content="提示：确保以正确的格式回答。将来源中提到的所有国家以大写字符形式返回。"
    ),
]

chain_prompt = ChatPromptTemplate(messages=prompt_messages)

qa_chain_pydantic = create_qa_with_structure_chain(
    llm, CustomResponseSchema, output_parser="pydantic", prompt=chain_prompt
)
final_qa_chain_pydantic = StuffDocumentsChain(
    llm_chain=qa_chain_pydantic,
    document_variable_name="context",
    document_prompt=doc_prompt,
)
retrieval_qa_pydantic = RetrievalQA(
    retriever=docsearch.as_retriever(), combine_documents_chain=final_qa_chain_pydantic
)
query = "What did he say about russia"
retrieval_qa_pydantic.run(query)

CustomResponseSchema(answer="He announced that American airspace will be closed off to all Russian flights, further isolating Russia and adding an additional squeeze on their economy. The Ruble has lost 30% of its value and the Russian stock market has lost 40% of its value. He also mentioned that Putin alone is to blame for Russia's reeling economy. The United States and its allies are providing support to Ukraine in their fight for freedom, including military, economic, and humanitarian assistance. The United States is giving more than $1 billion in direct assistance to Ukraine. He made it clear that American forces are not engaged and will not engage in conflict with Russian forces in Ukraine, but they are deployed to defend NATO allies in case Putin decides to keep moving west. He also mentioned that Putin's attack on Ukraine was premeditated and unprovoked, and that the West and NATO responded by building a coalition of freedom-loving nations to confront Putin. The free world is h